In [1]:
import pandas as pd
import requests
#import piexif - Do not need PIEXIF
from google.cloud import vision
from google.cloud.vision import types
from google.oauth2 import service_account
import os
import io
import json
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from bs4 import BeautifulSoup

In [2]:
FOLDER = '/Documents/Uploads'
APIKEY = pd.read_csv('/Users/anshul/Documents/APIKEYS/google_api_key.csv')
GOOGLE_SERVICE = '/Users/anshul/Downloads/My Project-93e8fa84a8bb.json'
N_ADDRESS = 4
trial_photo_location = 'Assets/trial.jpg'
initial_columns = ['GPSLongitude', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'DateTimeOriginal', 'PhotoName']
new_columns = ['GPSLongitude', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef']
disasters = pd.DataFrame(columns = initial_columns)
disasters2 = disasters.to_csv('disasters.csv', mode = 'a', header = True, index = False)

def get_exif_data(img):
    exif = Image.open(img)._getexif()
    if exif is not None:
        for key, value in exif.items():
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)
    else:
        return print("No GPS/Location Data Exists!")
    
    # Iphone
    if 'GPSInfo' in exif: #Exif/GPS data is lost when an image is uploaded to Slack, we NEED RAW data.
        for key in exif['GPSInfo'].keys():
            name = GPSTAGS.get(key, key)
            exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)
        exif_df = pd.DataFrame.from_dict(exif['GPSInfo'], orient='index') # Save to DataFrame from Dictionary
        exif_df = exif_df.T[new_columns]
        exif_datetime = pd.DataFrame.from_dict(exif, orient='index') # Isolate DateTimeOriginal from original exif dictionary
        exif_df['DateTimeOriginal'] = exif_datetime.T['DateTimeOriginal'] # Add new column to exif_df with DateTimeOriginal
        exif_df["PhotoName"] = img[7:]
        exif_df
        disasters.append(exif_df)
        return exif_df.to_csv('disasters.csv', mode='a', header=False, index=False) 
    
    # Android
    if 34853 in exif: #Exif data from Android phones
        for key in exif[34853].keys():
            name = GPSTAGS.get(key, key)
            exif[34853][name] = exif[34853].pop(key)
        exif_df_android = pd.DataFrame.from_dict(exif[34853], orient='index') 
        exif_df_android = exif_df_android.T[new_columns]
        exif_df_android['DateTimeOriginal'] = exif['DateTimeOriginal']
        exif_df_android["PhotoName"] = img[7:]
        exif_df_android
        disasters.append(exif_df_android)
        return exif_df_android.to_csv('disasters.csv', mode='a', header=False, index=False)

In [7]:
get_exif_data('Assets/trial2.jpg')

In [4]:
disasters

,GPSLongitude,GPSLatitudeRef,GPSLatitude,GPSLongitudeRef,DateTimeOriginal,PhotoName


In [8]:
disasters = pd.read_csv('disasters.csv')

In [9]:
disasters.head()

,GPSLongitude,GPSLatitudeRef,GPSLatitude,GPSLongitudeRef,DateTimeOriginal,PhotoName
0,"((122, 1), (22, 1), (1989, 100))",N,"((47, 1), (38, 1), (1033, 100))",W,2020:06:11 12:01:35,trial.jpg
1,"((135, 1), (46, 1), (5240, 100))",N,"((34, 1), (58, 1), (850, 100))",E,2019:10:15 04:43:23,trial2.jpg
